In [1]:
import sys
sys.path.append('robust_loss')

import robust_loss.adaptive 

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [15]:


class RobustAdaptativeLoss(object):
    
    def __init__(self):
        
        z = np.array([[0]])
        self.v_alpha = K.variable(z)

    def loss(self, y_true, y_pred, **kwargs):
          x = y_true - y_pred
    #x = K.reshape(x, shape=(-1, 1))
    
          loss = robust_loss.adaptive.AdaptiveLossFunction(num_channels=1,float_dtype="float32")
          alpha = loss.alpha()
          scale = loss.scale()
          op = K.update(self.v_alpha, alpha)
    # The alpha update must be part of the graph but it should
    # not influence the result.
          return loss(x) + 0 * op

    def alpha(self, y_true, y_pred):
        return self.v_alpha

def make_model():
    inp = Input(shape=(3,))
    out = Dense(1, use_bias=False)(inp)
    model = Model(inp, out)
    loss = RobustAdaptativeLoss()
    model.compile('adam', loss.loss, metrics=[loss.alpha])
    return model

model = make_model()
model.summary()



import numpy as np

FACTORS = np.array([0.5, 2.0, 5.0])
def target_fn(x):
    return np.dot(x, FACTORS.T)

N_SAMPLES=100
X = np.random.rand(N_SAMPLES, 3)
Y = np.apply_along_axis(target_fn, 1, X)


history = model.fit(X, Y, epochs=2, verbose=True)
print('final loss:', history.history['loss'][-1])

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Train on 100 samples
Epoch 1/2


FailedPreconditionError: Error while reading resource variable loss_7/dense_7_loss/LatentAlpha from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/loss_7/dense_7_loss/LatentAlpha/N10tensorflow3VarE does not exist.
	 [[{{node loss_7/dense_7_loss/Sigmoid/ReadVariableOp}}]]